In [1]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv('./datasets/breeds_info_clean.csv')
df


,breed,classification,obey,reps_lower,reps_upper,height_low_inches,height_high_inches,weight_low_lbs,weight_high_lbs
0,border collie,brightest dogs,0.95,1.0,4.0,19.0,21.0,40.0,40.0
1,poodle,brightest dogs,0.95,1.0,4.0,NaN,NaN,NaN,NaN
2,german shepherd,brightest dogs,0.95,1.0,4.0,22.0,26.0,75.0,90.0
3,golden retriever,brightest dogs,0.95,1.0,4.0,21.0,24.0,55.0,75.0
4,doberman pinscher,brightest dogs,0.95,1.0,4.0,26.0,28.0,60.0,100.0
...,...,...,...,...,...,...,...,...,...
163,miniature poodle,NaN,NaN,NaN,NaN,10.0,15.0,20.0,20.0
164,shetland sheepdog (sheltie),NaN,NaN,NaN,NaN,13.0,16.0,14.0,20.0
165,silky terrier,NaN,NaN,NaN,NaN,9.0,10.0,8.0,11.0
166,toy poodle,NaN,NaN,NaN,NaN,10.0,10.0,10.0,10.0


In [3]:
X = df[df['classification'].isna() == False]
X = X[X['height_low_inches'].isna() == False]
y = X['classification']
X = X.drop(columns = ['classification'])


In [4]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(X)
X = enc.transform(X).toarray()

In [5]:

X_train, X_test, y_train, y_test = train_test_split(X,y)

In [6]:
#knn classifier
knn = KNeighborsClassifier()
knn.fit(X_train,y_train)
knn.score(X_test, y_test)

1.0

In [7]:
K = df[df['classification'].isna() == True]
K = K.drop(columns = ['classification'])
K = enc.transform(K).toarray()

In [8]:
knn.predict(K)

array(['average working/obedience intelligence',
       'above average working dogs',
       'average working/obedience intelligence',
       'average working/obedience intelligence',
       'above average working dogs', 'above average working dogs',
       'average working/obedience intelligence',
       'above average working dogs',
       'average working/obedience intelligence',
       'above average working dogs', 'above average working dogs',
       'above average working dogs', 'above average working dogs',
       'excellent working dogs', 'average working/obedience intelligence',
       'average working/obedience intelligence',
       'average working/obedience intelligence',
       'above average working dogs', 'above average working dogs',
       'average working/obedience intelligence',
       'above average working dogs',
       'average working/obedience intelligence',
       'average working/obedience intelligence',
       'average working/obedience intelligence',
       

In [9]:
#logistic regression
log = LogisticRegression()
log.fit(X_train,y_train)
log.score(X_test,y_test)

1.0

In [10]:
for i in range(len(knn.predict(K))):
    if log.predict(K)[i] != knn.predict(K)[i]:
        print(i)

4
5
13
16
18


In [11]:
# naive bayes
from sklearn.naive_bayes import MultinomialNB
text_clf = MultinomialNB()
text_clf.fit(list(X_train), list(y_train))

text_clf.score(X_test,y_test)

1.0

In [12]:
for i in range(len(knn.predict(K))):
    if text_clf.predict(K)[i] != knn.predict(K)[i]:
        print(text_clf.predict(K)[i], '###########', knn.predict(K)[i])
        #print(i)

excellent working dogs ########### above average working dogs
average working/obedience intelligence ########### above average working dogs
above average working dogs ########### average working/obedience intelligence
fair working/obedience intelligence ########### above average working dogs
excellent working dogs ########### above average working dogs
fair working/obedience intelligence ########### average working/obedience intelligence


In [13]:
for i in range(len(knn.predict(K))):
    if log.predict(K)[i] != text_clf.predict(K)[i]:
        print(log.predict(K)[i], '###########', text_clf.predict(K)[i])
        #print(i)

average working/obedience intelligence ########### excellent working dogs
average working/obedience intelligence ########### excellent working dogs
above average working dogs ########### excellent working dogs
average working/obedience intelligence ########### fair working/obedience intelligence


In [14]:
final_list = list()
for i in range(len(knn.predict(K))):
    var_dict = dict()
    if knn.predict(K)[i] not in var_dict:
         var_dict[knn.predict(K)[i]] = 1
    else:
        var_dict[knn.predict(K)[i]] += 1
    if log.predict(K)[i] not in var_dict:
         var_dict[log.predict(K)[i]] = 1
    else:
        var_dict[log.predict(K)[i]] += 1
    if text_clf.predict(K)[i] not in var_dict:
         var_dict[text_clf.predict(K)[i]] = 1
    else:
        var_dict[text_clf.predict(K)[i]] += 1
    
    sorted_values = sorted(var_dict.values())
    if len(sorted_values) == 3:
        var_dict = {text_clf.predict(K)[i]:1}      
    #print(sorted_values, '###', i)
    for key in var_dict:
        if var_dict[key] == sorted_values[-1]:
            final_list.append(key)

In [15]:
K_1 = df[df['classification'].isna() == True]
K_1['classification'] = final_list
X_1 = df[df['classification'].isna() == False]
df_1 = pd.concat([K_1,X_1])

<ipython-input-15-98a2c704c23b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  K_1['classification'] = final_list


In [16]:
df_1

,breed,classification,obey,reps_lower,reps_upper,height_low_inches,height_high_inches,weight_low_lbs,weight_high_lbs
141,anatolian sheepdog,average working/obedience intelligence,NaN,NaN,NaN,27.0,29.0,100.0,150.0
142,great swiss mountain dog,above average working dogs,NaN,NaN,NaN,23.0,28.0,130.0,150.0
143,neopolitan mastiff,average working/obedience intelligence,NaN,NaN,NaN,24.0,30.0,100.0,150.0
144,beauceron,average working/obedience intelligence,NaN,NaN,NaN,24.0,27.0,100.0,120.0
145,black russian terrier,excellent working dogs,NaN,NaN,NaN,25.0,29.0,80.0,140.0
...,...,...,...,...,...,...,...,...,...
136,borzoi,lowest degree of working/obedience intelligence,0.1,81.0,100.0,26.0,28.0,70.0,100.0
137,chow chow,lowest degree of working/obedience intelligence,0.1,81.0,100.0,19.0,22.0,45.0,55.0
138,bulldog,lowest degree of working/obedience intelligence,0.1,81.0,100.0,12.0,16.0,50.0,60.0
139,basenji,lowest degree of working/obedience intelligence,0.1,81.0,100.0,17.0,17.0,20.0,22.0


In [17]:
df_1.to_csv('./datasets/breeds_info_clean_classifiers.csv', index=False)